# Carga de Datos

Hacemos distincción para posible estudio de rendimientos en la forma de guardado en disco y su lectura.

#### Carga Desde Binarios

In [1]:
\ts historical: get `:Data/DataWarehouse/Tables/historical

54 75499648


In [73]:
\ts resume_etf: get `:Data/DataWarehouse/Tables/resume_etf

0 4211024


#### Carga Desde Tablas Splayed

In [ ]:
\ts load `:Data/DataWarehouse/Tables/historicalSplayed/sym;
\ts historicalS: get `:Data/DataWarehouse/Tables/historicalSplayed/historical

In [ ]:
\ts load `:Data/DataWarehouse/Tables/resume_etfSplayed/sym;
\ts resume_etfS: get `:Data/DataWarehouse/Tables/resume_etfSplayed/resume_etf

# Cálculos

In [ ]:
meta historical

In [ ]:
select from historical where m200sma <> 0N, ticker=`EWJ, date>2020.01.01

In [ ]:
select days:count i by year: `year$date, ticker from historical

# Visualización de Datos

Para este apartado usaremos las sinergias entre KDB+/Q y Python, que al usar Jupyter Notebook se nos facilita aún más esa integración.

Usaremos el módulo y biblioteca por excelencia para esta representación gráfica en Python, [Matplotlib](https://matplotlib.org/stable/).

En primer lugar cargaremos en las variables **plt** y **mdates** los módulos de Python más usados para la representación gráfica y manipulación de fechas.

In [35]:
\l p.q
plt:.p.import[`matplotlib;`:pyplot]
mdates:.p.import[`matplotlib;`:dates]

Generamos las primeras funciones que repetiremos a la hora de generar gráficas, repitiendo código qSQL.

In [36]:
// HISTORICAL VALUE QUERIES

ax_x:{[ETF;DATEFILT]
    string each (select date from historical where ticker=ETF, date > DATEFILT)`date
 }
ax_y:{[ETF;DATEFILT]
    (select nav_value from historical where ticker=ETF, date > DATEFILT)`nav_value    
 }


// DAILY SMA QUERIES

aSMA_x:{[ETF;DATEFILT]
    string each (select date from historical where ticker=ETF, date > DATEFILT)`date
 }
aSMA_y:{[ETF;DATEFILT]
    (select d200sma from historical where ticker=ETF, date > DATEFILT)`d200sma    
 }


// MONTHLY SMA QUERIES

aMSMA_x:{[ETF;DATEFILT]
    string each (select date from historical where ticker=ETF, date > DATEFILT, m200sma<>0N)`date
 }
aMSMA_y:{[ETF;DATEFILT]
    (select m200sma from historical where ticker=ETF, date > DATEFILT, m200sma<>0N)`m200sma   
 }


// INFLOWS - OUTFLOWS % QUERIES

in_out_x:{[ETF;DATEFILT]
    string each (select date from historical where ticker=ETF, date > DATEFILT, in_out_flow_per<>0N)`date    
 }
in_out_y:{[ETF;DATEFILT]
    (select in_out_flow_per from historical where ticker=ETF, date > DATEFILT, in_out_flow_per<>0N)`in_out_flow_per
 }

### Generación de Gráficos de Históricos

In [37]:
historical_g:{[ETF;DATEFILTER]
    plt[`:figure][(`figsize);(22;14)]; 

    // Datos del gŕafico a representar
    plt[`:plot][ax_x[ETF;DATEFILTER];ax_y[ETF;DATEFILTER];"y-"];
    plt[`:plot][aSMA_x[ETF;DATEFILTER];aSMA_y[ETF;DATEFILTER];"r-"];
    plt[`:plot][aMSMA_x[ETF;DATEFILTER];aMSMA_y[ETF;DATEFILTER];"g-"];

    // Formato de fechas y eje X
    ax: plt[`:gca][];
    ax[`:xaxis][`:set_major_locator][(mdates[`:DayLocator][1])];
    plt[`:xticks][;][;][(`rotation;45)];
    plt[`:gcf][][`:autofmt_xdate][];
    
    name: string ETF;
    // Legenda y datos del gŕafico
    plt[`:title]["HISTORICAL",name];
    plt[`:xlabel]["DATE"];
    plt[`:ylabel]["NAV VALUE"];
    plt[`:legend][("NAV VALUE";"Daily SMA 200";"Monthly SMA 200")];
    
    //Guardado y mostrar por pantalla
    plt[`:savefig]["Data/DataWarehouse/Images/Hist/",name,".png"];
    plt[`:clf][];
 };

etfs: exec ticker from resume_etf
{historical_g[x; 2020.01.01]} each etfs;

### Generación de Gráficos de Volumen de Inversión

In [38]:
in_outflow_g:{[ETF;DATEFILTER]
    // Configuración del gráfico
    plt[`:figure][(`figsize);(14;10)];
    
    
    // Datos del gŕafico a representar
    plt[`:plot][in_out_x[ETF;DATEFILTER];in_out_y[ETF;DATEFILTER]; "b-o"];
    plt[`:axhline][(0)];
    
    // Formato de fechas y eje X
    ax: plt[`:gca][];
    ax[`:xaxis][`:set_major_locator][(mdates[`:DayLocator][1])];
    plt[`:xticks][;][;][(`rotation;45)];
    plt[`:gcf][][`:autofmt_xdate][];
    
    
    name: string ETF;
    // Legenda y datos del gŕafico
    plt[`:title]["% INFLOWS-OUTFLOWS ",name];
    plt[`:xlabel]["DATE"];
    plt[`:ylabel]["VALUE UP/DOWN"];
    
    //Guardado y mostrar por pantalla
    plt[`:savefig]["Data/DataWarehouse/Images/Inflow-Outflow/",name,".png"];;
    plt[`:clf][];
 }

etfs: exec ticker from resume_etf
{in_outflow_g[x; 2020.01.01]} each etfs;

# Modelo de Estrategia de Inversión en ETFs

En este apartado realizaremos el soporte para la elección de ETF.

En el anterior Notebook hemos realizado todos los cálculos necesarios para ahora poder selecionar y evaluar los ETF para nuestra cartera.

La técnica de selección, ponderación, etf es motivado por mi TFG de Administración de Empresas y los motivos de elección de cada apartado está explicado en él.


## Benchmark por Categoría

Se realizará el mismo benchmark para todos los ETF de la misma manera. Para poder conseguir el *Riesgo Diversificado* el mejor resultado por categoría:
> - **Mercado** en el que opera.
> - **Región** en la que opera.
> - **Clase de Equity** del que está compuesto el ETF.

## Benchmark para Modelo de Inversión

Se dará una valoración a cada elemento de:
- Tendencia del Mercado respecto a su SMA: **?%**
- Tendencia de Valoración respecto a su mSMA: **?%**
- Volatilidad a 1 año: **?%**
- Volatilidad a 6 meses: **?%**
- Volatilidad a 3 meses: **?%**
- Correlación Inflow/Outflow con Nav_Value: **?%**
- Distancia Temporal de su máximo + % valor actual respecto a él: **?%**
- Distancia Temporal de su mínimo + % valor actual respecto a él: **?%**
- Rendimiento los 5 años: **?%**
- Rendimiento los 1 año: **?%**
- Rendimiento los 6 meses: **?%**
- Drawdown a 5 años: **?%**
- Drawdown a 1 año: **?%**
- Drawdown a 6 mesess: **?%**

In [119]:
correlations_: {
    t: exec ticker from resume_etf;
    p: exec nav_value by ticker from historical where in_out_flow_per <> 0N;
    io: exec in_out_flow_per by ticker from historical where in_out_flow_per <> 0N;
    c: {p[x] cor io[x]} each t;
    flip `ticker`correlation!((tickers; c))

 }
correlations:{ [DATE]
    t: exec ticker from resume_etf;
    p: exec nav_value by ticker from historical where in_out_flow_per <> 0N, date > DATE;
    io: exec in_out_flow_per by ticker from historical where in_out_flow_per <> 0N, date > DATE;
    c: {p[x] cor io[x]} each t;
    flip `ticker`correlation!((tickers; c))
 }

In [125]:
table:()
table: update ticker: ticker from select ticker from resume_etf
table: update correlation: correlations_[][`correlation] from table

# Extras + info futuro


In [ ]:
count select ticker from resume_etf where market = `$"Developed"
select from historical where ticker = `EWA, m200sma <> 0N
count select ticker from resume_etf where market = `Emerging

//Filtrado  mezclando las dos tablas
select from historical where ticker in exec ticker from resume_etf where market = `$"Emerging"
//Lo mismo
show a: select ticker from resume_etf where market = `$"Developed"
select from historical where ticker in a[`ticker]

select distinct equity_class from resume_etf
select distinct region from resume_etf

//maximo aumento
select from historical where daily_retns=(max; daily_retns) fby ticker